In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/bank_authorizations.csv'

import pandas as pd
df = pd.read_csv(path)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-2899654490>:11: DtypeWarning: Columns (13,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,Fiscal Year,Unique Identifier,Deal Number,Decision,Decision Date,Effective Date,Expiration Date,Brokered,Deal Cancelled,Country,...,Working Capital Delegated Authority,Approved/Declined Amount,Disbursed/Shipped Amount,Undisbursed Exposure Amount,Outstanding Exposure Amount,Small Business Authorized Amount,Woman Owned Authorized Amount,Minority Owned Authorized Amount,Loan Interest Rate,Multiyear Working Capital Extension
0,2007.0,08003048XA0002,AP003048AA,Approved,11/29/2006,NaN,12/15/2011,No,No,Private Export Funding Corp.,...,NaN,24500000.0,24500000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,2007.0,08003048XA0003,AP003048AA,Approved,11/29/2006,11/29/2006,12/15/2016,No,No,Private Export Funding Corp.,...,NaN,50000000.0,50000000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,2007.0,08003048XA0004,AP003048AA,Approved,8/16/2007,8/16/2007,9/15/2017,No,No,Private Export Funding Corp.,...,NaN,136250000.0,136250000.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,2007.0,08064737XM0001,AP064737XM,Approved,9/21/2007,9/21/2007,7/31/2008,No,No,United States,...,Yes,2700000.0,2700000.0,0.0,0.0,2700000.0,0.0,2700000.0,NaN,NaN
4,2007.0,08069888XG0003,AP069888XG,Approved,11/24/2006,11/24/2006,1/1/2007,No,No,United States,...,Yes,233307.9,233307.9,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51032,2025.0,09778485ST1001,778485,Approved,12/20/2024,12/1/2024,12/1/2025,Yes,No,Australia,...,NaN,450000.0,0.0,450000.0,0.0,450000.0,450000.0,0.0,NaN,NaN
51033,2025.0,09778502ST1001,778502,Approved,12/11/2024,12/1/2024,12/1/2025,Yes,No,Finland,...,NaN,9000.0,0.0,9000.0,0.0,9000.0,0.0,0.0,NaN,NaN
51034,2025.0,09778606ST1001,778606,Approved,12/20/2024,11/1/2024,11/1/2025,Yes,No,Kuwait,...,NaN,49500.0,0.0,49500.0,0.0,49500.0,0.0,49500.0,NaN,NaN
51035,2025.0,09778722ST1001,778722,Approved,12/18/2024,1/1/2025,1/1/2026,Yes,No,United Arab Emirates,...,NaN,90000.0,0.0,90000.0,0.0,90000.0,0.0,90000.0,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: clean data
df['Decision_Binary'] = (df['Decision'].str.strip().str.lower() == 'approved').astype(int)

# Drop high-missing or irrelevant columns
high_missing_cols = df.columns[df.isnull().mean() > 0.5]
id_cols = ['Unique Identifier', 'Deal Number', 'Decision', 'Decision Date', 'Effective Date', 'Expiration Date']
drop_cols = list(high_missing_cols) + id_cols
df_model = df.drop(columns=drop_cols, errors='ignore')

# Identify low-cardinality categorical columns
cat_cols = df_model.select_dtypes(include='object').columns
low_card_cols = [col for col in cat_cols if df_model[col].nunique() <= 100]

# Select features
numeric_cols = df_model.select_dtypes(include=np.number).columns.drop('Decision_Binary', errors='ignore').tolist()
selected_columns = numeric_cols + low_card_cols + ['Decision_Binary']
df_reduced = df_model[selected_columns].copy()

# Encode categoricals
df_reduced[low_card_cols] = df_reduced[low_card_cols].fillna('Missing')
df_encoded = pd.get_dummies(df_reduced, columns=low_card_cols, drop_first=True)
df_encoded = df_encoded.fillna(0)

# Split into features and target
X = df_encoded.drop(columns='Decision_Binary')
y = df_encoded['Decision_Binary']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Show results
target_names = ['Disapproved', 'Approved']  # Class 0 = Disapproved, Class 1 = Approved

print("Model has a fallacy pitfall. The Dissaproved transactions are not being accounted for.")
print(classification_report(y_test, y_pred, target_names=target_names))


Model has a fallacy pitfall. The Dissaproved transactions are not being accounted for.
              precision    recall  f1-score   support

 Disapproved       0.00      0.00      0.00         2
    Approved       1.00      1.00      1.00     10206

    accuracy                           1.00     10208
   macro avg       0.50      0.50      0.50     10208
weighted avg       1.00      1.00      1.00     10208



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: clean data
df['Decision_Binary'] = (df['Decision'].str.strip().str.lower() == 'approved').astype(int)

# Drop high-missing or irrelevant columns
high_missing_cols = df.columns[df.isnull().mean() > 0.5]
id_cols = ['Unique Identifier', 'Deal Number', 'Decision', 'Decision Date', 'Effective Date', 'Expiration Date']
drop_cols = list(high_missing_cols) + id_cols
df_model = df.drop(columns=drop_cols, errors='ignore')

# Identify low-cardinality categorical columns
cat_cols = df_model.select_dtypes(include='object').columns
low_card_cols = [col for col in cat_cols if df_model[col].nunique() <= 100]

# Select features
numeric_cols = df_model.select_dtypes(include=np.number).columns.drop('Decision_Binary', errors='ignore').tolist()
selected_columns = numeric_cols + low_card_cols + ['Decision_Binary']
df_reduced = df_model[selected_columns].copy()

# Encode categoricals
df_reduced[low_card_cols] = df_reduced[low_card_cols].fillna('Missing')
df_encoded = pd.get_dummies(df_reduced, columns=low_card_cols, drop_first=True)
df_encoded = df_encoded.fillna(0)

# Split into features and target
X = df_encoded.drop(columns='Decision_Binary')
y = df_encoded['Decision_Binary']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Train logistic regression model with class balancing
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Show results
target_names = ['Disapproved', 'Approved']  # Class 0 = Disapproved, Class 1 = Approved
print("Model is accurate and is now picking up the Dissapproved transactions.")
print(classification_report(y_test, y_pred, target_names=target_names))


Model is now picking up the Dissapproved transactions.
              precision    recall  f1-score   support

 Disapproved       0.01      1.00      0.02         2
    Approved       1.00      0.98      0.99     10206

    accuracy                           0.98     10208
   macro avg       0.50      0.99      0.50     10208
weighted avg       1.00      0.98      0.99     10208

